# Trabajo Final - Tostadora "Campesino"

## Autor: Leonard David Vivas Dallos

## Objetivo
Predecir la calidad del café basados en el Puntaje de Taza.

## Indicaciones

1. Analizar los datos y definir las variables que se utilizarán de input. Tenga cuidad que no se filtre información al momento de elegirlas.
2. Definir el preprocesamiento que se debe hacer tanto para las X como para la Y.
3. Entrenar al menos dos modelos de regresión.
4. Utilizar un método de explicabilidad (LIME, SHAP, Feature Importance)
5. Reportar resultados.

El paso inicial es cargar los datos y realizar un análisis exploratorio para entender las variables disponibles y su relación con el Puntaje de Taza. Como sabemos, la información brindada para este análisis viene en un archivo de Excel, .xlsx, que contiene la información necesaria para el análisis. En primer lugar, cargaremos los datos, y los revisaremos para identificar las variables relevantes y como podemos relacionarlas con el Puntaje de Taza.

In [365]:
# Importamos las librerías necesarias para el análisis de datos y visualización
import pandas as pd                # Librería para manipulación y análisis de datos (estructuras tipo DataFrame)
import numpy as np                 # Librería para operaciones numéricas y matrices
import matplotlib.pyplot as plt    # Librería para generar gráficos estáticos
import seaborn as sns              # Librería basada en matplotlib para visualización estadística más atractiva
import plotly.express as px        # Librería para crear gráficos interactivos y visualizaciones avanzadas
from sklearn.preprocessing import LabelEncoder, MinMaxScaler  # Importamos LabelEncoder para codificar variables categóricas

# Configuraciones adicionales para mejorar la interacción y visualización
plt.style.use('ggplot')            # Establece el estilo de los gráficos a 'ggplot' para una estética tipo R
%matplotlib inline
pd.set_option('display.max_columns', None)  # Evita el truncamiento de columnas en la visualización de datos

### Archivo "CC FT 17   Formato de Control de Calidad Café de Trillado (1).xlsx"

In [366]:
# Inicialmente, cargamos los datos desde el archivo Excel
data_control_calidad = pd.read_excel("CC FT 17   Formato de Control de Calidad Café de Trillado (1).xlsx", skiprows=7)
# Revisamos las primeras filas del DataFrame para entender su estructura
data_control_calidad.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,%,C/NC,#,C/NC.1,C/NC.2,Unnamed: 9,N°,C/NC.3,SI/NO,Unnamed: 13
0,19-07-22,01-190722,Madre Laura,765.00,10.9,C,14.0,C,C,"Chocolate negro, toque frutal, cuerpo medio, a...",84,C,Si,LFQ
1,19-7-22,09-190722,Tabi Natural,204.00,10.2,C,14.0,C,C,"Frutas maduras, nibs de cacao, acidez brillant...",85,C,Si,LFQ
2,19-07-22,10-190722,Don Mario,165.00,10.7,C,14.0,C,C,"Panela, durazno, miel, acidez brillante citric...",84.5,C,Si,LFQ
3,27-07-22,07-19-07-22,Don Felix,0.45,10.5,C,14.0,C,C,"Moras maduras, chocolate negro, acidez media c...",84.5,C,Si,LFQ
4,31-10-22,01-291022,Madre Laura,105.00,10.7,C,14.0,C,C,"Chocolate negro, toque frutal, cuerpo medio, a...",84,C,Si,LFQ


Dado un vistazo inicial a los datos, por la configuración del archivo de Excel, tenemos que saltar las primeras 7 filas para obtener el DataFrame correctamente, sin embargo, aún tendremos que asignar manualmente los nombres de las columnas, ya que el archivo no los contiene en la fila 8 (por problemas de formato de filas combinadas).

In [367]:
# Asignamos manualmente los nombres de las columnas
data_control_calidad.columns = [
    'Fecha', 'Lote', 'Denominación', 'Cantidad', '% Humedad', 'C/NC Humedad', 'Mallas', 'C/NC Mallas', 'Verificación Física Café',
    'Notas de Catación', 'Puntaje de Taza', 'C/NC Puntaje de Taza', 'Liberación de Lote(Si/No)', 'Responsable'
]
# Revisamos nuevamente las primeras filas del DataFrame con los nombres de columnas actualizados
data_control_calidad.head()

,Fecha,Lote,Denominación,Cantidad,% Humedad,C/NC Humedad,Mallas,C/NC Mallas,Verificación Física Café,Notas de Catación,Puntaje de Taza,C/NC Puntaje de Taza,Liberación de Lote(Si/No),Responsable
0,19-07-22,01-190722,Madre Laura,765.00,10.9,C,14.0,C,C,"Chocolate negro, toque frutal, cuerpo medio, a...",84,C,Si,LFQ
1,19-7-22,09-190722,Tabi Natural,204.00,10.2,C,14.0,C,C,"Frutas maduras, nibs de cacao, acidez brillant...",85,C,Si,LFQ
2,19-07-22,10-190722,Don Mario,165.00,10.7,C,14.0,C,C,"Panela, durazno, miel, acidez brillante citric...",84.5,C,Si,LFQ
3,27-07-22,07-19-07-22,Don Felix,0.45,10.5,C,14.0,C,C,"Moras maduras, chocolate negro, acidez media c...",84.5,C,Si,LFQ
4,31-10-22,01-291022,Madre Laura,105.00,10.7,C,14.0,C,C,"Chocolate negro, toque frutal, cuerpo medio, a...",84,C,Si,LFQ


In [368]:
data_control_calidad['Fecha'] = pd.to_datetime(data_control_calidad['Fecha'], dayfirst=True, errors='coerce')

data_control_calidad.head()

C:\Users\David\AppData\Local\Temp\ipykernel_24788\640125936.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data_control_calidad['Fecha'] = pd.to_datetime(data_control_calidad['Fecha'], dayfirst=True, errors='coerce')


,Fecha,Lote,Denominación,Cantidad,% Humedad,C/NC Humedad,Mallas,C/NC Mallas,Verificación Física Café,Notas de Catación,Puntaje de Taza,C/NC Puntaje de Taza,Liberación de Lote(Si/No),Responsable
0,2022-07-19,01-190722,Madre Laura,765.00,10.9,C,14.0,C,C,"Chocolate negro, toque frutal, cuerpo medio, a...",84,C,Si,LFQ
1,2022-07-19,09-190722,Tabi Natural,204.00,10.2,C,14.0,C,C,"Frutas maduras, nibs de cacao, acidez brillant...",85,C,Si,LFQ
2,2022-07-19,10-190722,Don Mario,165.00,10.7,C,14.0,C,C,"Panela, durazno, miel, acidez brillante citric...",84.5,C,Si,LFQ
3,2022-07-27,07-19-07-22,Don Felix,0.45,10.5,C,14.0,C,C,"Moras maduras, chocolate negro, acidez media c...",84.5,C,Si,LFQ
4,2022-10-31,01-291022,Madre Laura,105.00,10.7,C,14.0,C,C,"Chocolate negro, toque frutal, cuerpo medio, a...",84,C,Si,LFQ


#### Archivo "CC FT 18  Formato de  Tostión (1).xlsx"

In [369]:
data_tostion = pd.read_excel("CC FT 18  Formato de  Tostión (1).xlsx", skiprows=5)

data_tostion.head()

,Fecha,Lote,Origen,Variedad,Proceso,Beneficio,Peso en Verde,Merma,Peso en Tostado,Perfil,Temp. De inicio y final,Tiempo de tueste,Observaciones,Tostador
0,22-07-22,01-190722,Jerico,Dos mil,Tradicional,Lavado,9.00,15.000000,7.65,Filtrado,175°/191°,08:01:00,NaN,LFQ
1,25-07-22,01-190722,Jerico,Dos mil,Tradicional,Lavado,3.00,16.666667,2.50,Espressso,175°/195°,08:42:00,NaN,LFQ
2,25-07-22,01-190722,Jerico,Dos mil,Tradicional,Lavado,9.00,16.444444,7.52,Filtrado,175°/190°,07:58:00,NaN,LFQ
3,28-07-22,01-190722,Jerico,Dos mil,Tradicional,Lavado,11.70,16.324786,9.79,Filtrado,175°/191°,08:02:00,NaN,LFQ
4,28-07-22,09-190722,Ciudad Bolivar,Tabi,Natural,Natural,0.45,24.444444,0.34,Filtrado,150°/186°,08:10:00,NaN,LFQ


Podemos ver que la columna de "Temp. De inicio y final" contiene dos valores separados por un /, lo que indica que debemos dividir esta columna en dos columnas separadas: "Temp. De inicio" y "Temp. Final". Esto nos permitirá trabajar con los datos de temperatura de manera más efectiva.

In [370]:
# Dividimos la columna "Temp. De inicio y final" en dos columnas separadas
data_tostion[['Temp. De inicio', 'Temp. Final']] = data_tostion['Temp. De inicio y final'].str.split('/', expand=True)
# Eliminamos la columna original "Temp. De inicio y final"
data_tostion.drop(columns=['Temp. De inicio y final'], inplace=True)
# Revisamos las primeras filas del DataFrame de tostión
data_tostion.head()

,Fecha,Lote,Origen,Variedad,Proceso,Beneficio,Peso en Verde,Merma,Peso en Tostado,Perfil,Tiempo de tueste,Observaciones,Tostador,Temp. De inicio,Temp. Final
0,22-07-22,01-190722,Jerico,Dos mil,Tradicional,Lavado,9.00,15.000000,7.65,Filtrado,08:01:00,NaN,LFQ,175°,191°
1,25-07-22,01-190722,Jerico,Dos mil,Tradicional,Lavado,3.00,16.666667,2.50,Espressso,08:42:00,NaN,LFQ,175°,195°
2,25-07-22,01-190722,Jerico,Dos mil,Tradicional,Lavado,9.00,16.444444,7.52,Filtrado,07:58:00,NaN,LFQ,175°,190°
3,28-07-22,01-190722,Jerico,Dos mil,Tradicional,Lavado,11.70,16.324786,9.79,Filtrado,08:02:00,NaN,LFQ,175°,191°
4,28-07-22,09-190722,Ciudad Bolivar,Tabi,Natural,Natural,0.45,24.444444,0.34,Filtrado,08:10:00,NaN,LFQ,150°,186°


Adicionalmente, revisemos los nombres de las columnas del archivo de tostión para asegurarnos de que están correctamente asignados y no contienen espacios adicionales o caracteres no deseados.

In [371]:
print(data_tostion.columns.tolist())

['Fecha ', 'Lote ', 'Origen', 'Variedad', 'Proceso', 'Beneficio ', 'Peso en Verde', 'Merma ', 'Peso en Tostado ', 'Perfil ', 'Tiempo de tueste', 'Observaciones ', 'Tostador', 'Temp. De inicio', 'Temp. Final']


Como presentiamos, las columnas tienen espacios adicionales, lo que puede causar problemas al acceder a ellas. Por lo tanto, es importante eliminar estos espacios para evitar errores en el análisis posterior. Aparte de hacer esto, formatearemos las fechas para que todas tengan el mismo formato, lo que facilitará su manejo y análisis posterior.

In [372]:
data_tostion.columns = data_tostion.columns.str.strip()

data_tostion['Fecha'] = pd.to_datetime(data_tostion['Fecha'], dayfirst=True, errors='coerce')

data_tostion.head()

C:\Users\David\AppData\Local\Temp\ipykernel_24788\1680212727.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data_tostion['Fecha'] = pd.to_datetime(data_tostion['Fecha'], dayfirst=True, errors='coerce')


,Fecha,Lote,Origen,Variedad,Proceso,Beneficio,Peso en Verde,Merma,Peso en Tostado,Perfil,Tiempo de tueste,Observaciones,Tostador,Temp. De inicio,Temp. Final
0,2022-07-22,01-190722,Jerico,Dos mil,Tradicional,Lavado,9.00,15.000000,7.65,Filtrado,08:01:00,NaN,LFQ,175°,191°
1,2022-07-25,01-190722,Jerico,Dos mil,Tradicional,Lavado,3.00,16.666667,2.50,Espressso,08:42:00,NaN,LFQ,175°,195°
2,2022-07-25,01-190722,Jerico,Dos mil,Tradicional,Lavado,9.00,16.444444,7.52,Filtrado,07:58:00,NaN,LFQ,175°,190°
3,2022-07-28,01-190722,Jerico,Dos mil,Tradicional,Lavado,11.70,16.324786,9.79,Filtrado,08:02:00,NaN,LFQ,175°,191°
4,2022-07-28,09-190722,Ciudad Bolivar,Tabi,Natural,Natural,0.45,24.444444,0.34,Filtrado,08:10:00,NaN,LFQ,150°,186°


Observemos que la columna "Observaciones" al parecer solo contiene NaN, lo que indica que no hay información en esta. Confirmemos esto y, si es así, eliminaremos esta columna del DataFrame para evitar confusiones en el análisis posterior.

In [373]:
# Revisar valores de "Observaciones" para confirmar si contiene solo NaN
observaciones_nulos = data_tostion['Observaciones'].isnull().all()
if observaciones_nulos:
    # Si la columna "Observaciones" contiene solo NaN, la eliminamos
    data_tostion.drop(columns=['Observaciones'], inplace=True)
else:
    print("La columna 'Observaciones' contiene datos y no se eliminará.")

In [374]:
data_tostion.head()

,Fecha,Lote,Origen,Variedad,Proceso,Beneficio,Peso en Verde,Merma,Peso en Tostado,Perfil,Tiempo de tueste,Tostador,Temp. De inicio,Temp. Final
0,2022-07-22,01-190722,Jerico,Dos mil,Tradicional,Lavado,9.00,15.000000,7.65,Filtrado,08:01:00,LFQ,175°,191°
1,2022-07-25,01-190722,Jerico,Dos mil,Tradicional,Lavado,3.00,16.666667,2.50,Espressso,08:42:00,LFQ,175°,195°
2,2022-07-25,01-190722,Jerico,Dos mil,Tradicional,Lavado,9.00,16.444444,7.52,Filtrado,07:58:00,LFQ,175°,190°
3,2022-07-28,01-190722,Jerico,Dos mil,Tradicional,Lavado,11.70,16.324786,9.79,Filtrado,08:02:00,LFQ,175°,191°
4,2022-07-28,09-190722,Ciudad Bolivar,Tabi,Natural,Natural,0.45,24.444444,0.34,Filtrado,08:10:00,LFQ,150°,186°


#### Archivo "CC FT 21   Formato de Control de Despachos (1).xlsx"

In [375]:
data_despachos = pd.read_excel("CC FT 21   Formato de Control de Despachos (1).xlsx", skiprows=5)

# Eliminamos la primera fila que contiene el encabezado duplicado
data_despachos = data_despachos.iloc[1:]

data_despachos.head()

,# PEDIDO,FECHA DE TUESTE,FECHA DE EMPAQUE,TIPO DE CAFÉ,PRESENTACIÓN,Unnamed: 5,CANTIDAD,Unnamed: 7,Unnamed: 8,CLIENTE,RESPONSABLE DESPACHO,VERIFICA
1,564,22-7-22,25-7-22,Madre Laura,500 Gr,NaN,3.0,NaN,NaN,Natalia Londoño,LFQ,MS
2,564,22-7-22,25-7-22,Madre Laura,1000 Gr,NaN,1.0,NaN,NaN,Natalia Londoño,LFQ,MS
3,566,22-7-22,25-7-22,Madre Laura,2500 Gr,NaN,2.0,NaN,NaN,Mateo Duque,LFQ,MS
4,567,22-7-22,26-7-22,Madre Laura,2500 Gr,NaN,1.0,NaN,NaN,Akio,LFQ,MS
5,568,25-7-22,26-7-22,Madre Laura,2500 Gr,NaN,3.0,NaN,NaN,Juan Camilo Baez,LFQ,MS


En un primer vistazo, podemos ver que tenemos ciertas columnas que no traen información alguna, esto debido al manejo de las columnas combinadas en el archivo de Excel. Por lo tanto, debemos eliminar las columnas que no contienen información relevante para nuestro análisis.

In [376]:
# Eliminamos las columnas con nombre Unnamed que no contienen información relevante
data_despachos = data_despachos.loc[:, ~data_despachos.columns.str.contains('^Unnamed')]
# Revisamos las primeras filas del DataFrame de despachos
data_despachos.head()

,# PEDIDO,FECHA DE TUESTE,FECHA DE EMPAQUE,TIPO DE CAFÉ,PRESENTACIÓN,CANTIDAD,CLIENTE,RESPONSABLE DESPACHO,VERIFICA
1,564,22-7-22,25-7-22,Madre Laura,500 Gr,3.0,Natalia Londoño,LFQ,MS
2,564,22-7-22,25-7-22,Madre Laura,1000 Gr,1.0,Natalia Londoño,LFQ,MS
3,566,22-7-22,25-7-22,Madre Laura,2500 Gr,2.0,Mateo Duque,LFQ,MS
4,567,22-7-22,26-7-22,Madre Laura,2500 Gr,1.0,Akio,LFQ,MS
5,568,25-7-22,26-7-22,Madre Laura,2500 Gr,3.0,Juan Camilo Baez,LFQ,MS


Revisemos si en este caso tenemos el mismo comportamiento de columnas que en el archivo de tostión, es decir, si las columnas tienen espacios adicionales o caracteres no deseados. Si es así, debemos limpiarlas para evitar problemas al acceder a ellas. Además, debemos asegurarnos de que las fechas estén en un formato consistente para facilitar su manejo y análisis posterior.

In [377]:
print(data_despachos.columns.tolist())

['# PEDIDO ', 'FECHA DE TUESTE ', 'FECHA DE EMPAQUE ', 'TIPO DE CAFÉ   ', 'PRESENTACIÓN ', 'CANTIDAD ', 'CLIENTE ', 'RESPONSABLE DESPACHO ', 'VERIFICA ']


In [378]:
data_despachos.columns = data_despachos.columns.str.strip()

data_despachos['FECHA DE TUESTE'] = pd.to_datetime(data_despachos['FECHA DE TUESTE'], dayfirst=True, errors='coerce')
data_despachos['FECHA DE EMPAQUE'] = pd.to_datetime(data_despachos['FECHA DE EMPAQUE'], dayfirst=True, errors='coerce')

data_despachos.head()

C:\Users\David\AppData\Local\Temp\ipykernel_24788\376962012.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data_despachos['FECHA DE TUESTE'] = pd.to_datetime(data_despachos['FECHA DE TUESTE'], dayfirst=True, errors='coerce')
C:\Users\David\AppData\Local\Temp\ipykernel_24788\376962012.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data_despachos['FECHA DE EMPAQUE'] = pd.to_datetime(data_despachos['FECHA DE EMPAQUE'], dayfirst=True, errors='coerce')


,# PEDIDO,FECHA DE TUESTE,FECHA DE EMPAQUE,TIPO DE CAFÉ,PRESENTACIÓN,CANTIDAD,CLIENTE,RESPONSABLE DESPACHO,VERIFICA
1,564,2022-07-22,2022-07-25,Madre Laura,500 Gr,3.0,Natalia Londoño,LFQ,MS
2,564,2022-07-22,2022-07-25,Madre Laura,1000 Gr,1.0,Natalia Londoño,LFQ,MS
3,566,2022-07-22,2022-07-25,Madre Laura,2500 Gr,2.0,Mateo Duque,LFQ,MS
4,567,2022-07-22,2022-07-26,Madre Laura,2500 Gr,1.0,Akio,LFQ,MS
5,568,2022-07-25,2022-07-26,Madre Laura,2500 Gr,3.0,Juan Camilo Baez,LFQ,MS


### Análisis de los archivos

Ya obtenidos los DataFrames de los archivos de Excel, y limpiados los nombres de las columnas y los tipos de algunas columnas, procedemos a realizar un análisis exploratorio de los datos, entre estos, revisaremos a que hace referencia cada columna, si tiene información relevante, si es necesario eliminarla o transformarla, y si es necesario realizar algún tipo de preprocesamiento adicional. Adicionalmente, revisaremos cual de esta información es posible usar para nuestro objetivo de predecir el Puntaje de Taza, para así evitar filtrar información al momento de elegir las variables de entrada (X) y la variable objetivo (Y).

Comencemos revisando cada uno de los archivos, y expliquemos que información contiene cada uno de ellos, y como podemos relacionarlos entre sí para obtener la información necesaria para nuestro análisis.

**Archivo "CC FT 17   Formato de Control de Calidad Café de Trillado (1).xlsx"**

| Columna                              | Descripción                                                                                            |
| ------------------------------------ | ------------------------------------------------------------------------------------------------------ |
| **Fecha**                            | Fecha en que se realiza el control de calidad del lote.                                                |
| **Lote**                             | Identificador único del lote evaluado. Permite la trazabilidad con los otros formatos.                 |
| **Denominación**                     | Nombre comercial o tipo de café dentro del lote.                                                       |
| **Cantidad**                         | Cantidad total de café evaluado (probablemente en kg).                                                 |
| **%H**                               | Porcentaje de humedad del café. Se evalúa si cumple (C) o no cumple (NC) con los estándares.           |
| **Mallas**                           | Tamaño de grano evaluado con tamices (mallas). Se indica el número de malla y si cumple (C) o no (NC). |
| **Verificación Física Café Tostado** | Inspección física del grano tostado, indicando si cumple con las características deseadas (N/NC).      |
| **Notas de Catación**                | Comentarios cualitativos del catador sobre aroma, sabor, acidez, cuerpo, etc.                          |
| **Puntaje**                          | Evaluación cuantitativa del café en taza (SCA score). Se indica el número y si cumple (C/NC).          |
| **Liberación de lote**               | Indica si el lote fue aprobado para comercialización (Sí / No).                                        |
| **Responsable**                      | Nombre de la persona que realizó la evaluación.                                                        |



**Archivo "CC FT 18  Formato de  Tostión (1).xlsx"**

| Columna              | Descripción                                                                              |
| -------------------- | ---------------------------------------------------------------------------------------- |
| **Fecha**            | Fecha en que se realizó la tostión del lote.                                             |
| **Lote**             | Identificador del lote, que debería coincidir con el de calidad.                         |
| **Origen**           | Lugar o finca de donde proviene el café verde. Influye en las propiedades sensoriales.   |
| **Variedad**         | Tipo de grano (Ej: Caturra, Castillo, Geisha, etc). Importante para el perfil sensorial. |
| **Proceso**          | Proceso postcosecha (lavado, natural, honey, etc). Afecta sabor, acidez, cuerpo.         |
| **Beneficio**        | Método de beneficio húmedo o seco (posiblemente nombre de la estación de beneficio).     |
| **Peso en Verde**    | Peso del café antes de tostar.                                                           |
| **Merma**            | Pérdida de peso durante el tostado (por evaporación). Importante para rendimiento.       |
| **Peso en Tostado**  | Peso final del lote después del proceso de tostión.                                      |
| **Perfil**           | Perfil de tostión buscado (claro, medio, oscuro). Determina la expresión de sabores.     |
| **Temp. de Inicio**  | Temperatura inicial del tambor o tostadora al empezar.                                   |
| **Temp. de Final**   | Temperatura al final del proceso de tostión.                                             |
| **Tiempo de Tueste** | Duración total del proceso de tostión (en minutos).                                      |
| **Tostador**         | Nombre del operario o responsable de la tostión.                                         |


**Archivo "CC FT 21   Formato de Control de Despachos (1).xlsx"**

| Columna                  | Descripción                                                                    |
| ------------------------ | ------------------------------------------------------------------------------ |
| **# Pedido**             | Número de pedido asignado para el despacho del café.                           |
| **Fecha de Tueste**      | Fecha en la que se realizó el tostado del café despachado. Útil para frescura. |
| **Fecha de Empaque**     | Fecha en la que se empacó el producto final. También influye en la frescura.   |
| **Tipo de Café**         | Clasificación del café (especial, premium, estándar, etc.).                    |
| **Presentación**         | Formato de entrega (molido, en grano, en bolsa de 250g, etc.).                 |
| **Cantidad**             | Cantidad de café despachado.                                                   |
| **Cliente**              | Nombre del cliente o entidad a la que se realizó el despacho.                  |
| **Responsable Despacho** | Nombre del encargado del despacho.                                             |
| **Quién verifica**       | Persona que valida el despacho antes de salir.                                 |


Una vez tenemos la información de las columnas, es importante detectar cuales son las variables que son útiles para determinar el Puntaje de Taza, pero más importante aún, es detectar si tenemos información que no tenemos disponible al momento de hacer la predicción, ya que esto nos permitirá definir las variables de entrada (X) y la variable objetivo (Y) de nuestro modelo de regresión. Para esto, según el análisis de las columnas en el contexto en que estamos, podemos definir las siguientes variables:

**Archivo "CC FT 17   Formato de Control de Calidad Café de Trillado (1).xlsx"**

En cuanto a este archivo, se puede notar que tenemos la variable a predecir, el Puntaje de Taza, que se encuentra en la columna "Puntaje". Por otro lado, notemos que tenemos algunas columnas que no tenemos a la mano al momento del análisis

In [379]:
data_control_calidad.head()

,Fecha,Lote,Denominación,Cantidad,% Humedad,C/NC Humedad,Mallas,C/NC Mallas,Verificación Física Café,Notas de Catación,Puntaje de Taza,C/NC Puntaje de Taza,Liberación de Lote(Si/No),Responsable
0,2022-07-19,01-190722,Madre Laura,765.00,10.9,C,14.0,C,C,"Chocolate negro, toque frutal, cuerpo medio, a...",84,C,Si,LFQ
1,2022-07-19,09-190722,Tabi Natural,204.00,10.2,C,14.0,C,C,"Frutas maduras, nibs de cacao, acidez brillant...",85,C,Si,LFQ
2,2022-07-19,10-190722,Don Mario,165.00,10.7,C,14.0,C,C,"Panela, durazno, miel, acidez brillante citric...",84.5,C,Si,LFQ
3,2022-07-27,07-19-07-22,Don Felix,0.45,10.5,C,14.0,C,C,"Moras maduras, chocolate negro, acidez media c...",84.5,C,Si,LFQ
4,2022-10-31,01-291022,Madre Laura,105.00,10.7,C,14.0,C,C,"Chocolate negro, toque frutal, cuerpo medio, a...",84,C,Si,LFQ


In [380]:
# Impresión de distintos responsables
responsables = data_control_calidad['Responsable'].unique()
print("Responsables únicos en el archivo de control de calidad:")
for responsable in responsables:
    print(responsable)

print(responsables)


Responsables únicos en el archivo de control de calidad:
LFQ
LFQ 
nan
['LFQ' 'LFQ ' nan]


Para este archivo, notemos que solo tenemos un responsable, lo que indica que no es información que aporte mucho de momento al análisis. Sin embargo, revisarlo nos permite identificar un problema que debemos tener en cuenta y es que es posible que mismos valores esten difiriendo en nuestro análisis debido a errores de tipeo o introducción de espacios en el mapeo. Sin embargo, analizaremos esto más adelante, cuando tengamos los datos consolidados.

Las notas de catación son una columna que contiene información cualitativa, que es bastante difícil de manejar, por lo que la descartaremos de momento. Por otro lado, la columna "C/NC Puntaje de Taza" y "Liberación de lote" son columnas que están derivadas de nuestra variable objetivo, por lo que no las utilizaremos como variables de entrada. Finalmente, la columna "Responsable" es una columna que no aporta información relevante para nuestro análisis, ya que solo tenemos un responsable, por lo que la descartaremos de momento.

In [381]:
# Eliminamos las columnas que no aportan información relevante
data_control_calidad = data_control_calidad.drop(columns=['Responsable', 'Notas de Catación', 'C/NC Puntaje de Taza', 'Liberación de Lote(Si/No)'])
# Revisamos las primeras filas del DataFrame de control de calidad
data_control_calidad.head()

,Fecha,Lote,Denominación,Cantidad,% Humedad,C/NC Humedad,Mallas,C/NC Mallas,Verificación Física Café,Puntaje de Taza
0,2022-07-19,01-190722,Madre Laura,765.00,10.9,C,14.0,C,C,84
1,2022-07-19,09-190722,Tabi Natural,204.00,10.2,C,14.0,C,C,85
2,2022-07-19,10-190722,Don Mario,165.00,10.7,C,14.0,C,C,84.5
3,2022-07-27,07-19-07-22,Don Felix,0.45,10.5,C,14.0,C,C,84.5
4,2022-10-31,01-291022,Madre Laura,105.00,10.7,C,14.0,C,C,84


**Archivo "CC FT 18  Formato de  Tostión (1).xlsx"**

Para este archivo, notemos que tenemos la información general del proceso de tostión para el lote, lo que nos permitirá relacionar este proceso con el Puntaje de Taza. Sin embargo, revisemos cuantos tostadores tenemos, ya que si tenemos más de uno, debemos tener en cuenta que cada uno puede tener un perfil de tostión diferente, lo que puede afectar el Puntaje de Taza. Por lo tanto, debemos revisar si tenemos información suficiente para relacionar el Puntaje de Taza con el proceso de tostión.

In [382]:
# Verificamos si tenemos más de un tostador
tostadores = data_tostion['Tostador'].unique()
if len(tostadores) > 1:
    print(f"Tenemos {len(tostadores)} tostadores diferentes: {', '.join(tostadores)}")
    

Tenemos 2 tostadores diferentes: LFQ, LFQ 


Nuevamente, tenemos el mismo fenómeno de espacios en los nombres de las columnas, sin embargo, solo tenemos un tostador, por lo que lo podemos omitir de momento, el resto de columnas son relevantes para nuestro análisis, ya que nos permiten relacionar el proceso de tostión con el Puntaje de Taza. Por lo tanto, las utilizaremos como variables de entrada.

In [383]:
# Eliminamos las columnas que no aportan información relevante
data_tostion = data_tostion.drop(columns=['Tostador'])
# Revisamos las primeras filas del DataFrame de tostión
data_tostion.head()

,Fecha,Lote,Origen,Variedad,Proceso,Beneficio,Peso en Verde,Merma,Peso en Tostado,Perfil,Tiempo de tueste,Temp. De inicio,Temp. Final
0,2022-07-22,01-190722,Jerico,Dos mil,Tradicional,Lavado,9.00,15.000000,7.65,Filtrado,08:01:00,175°,191°
1,2022-07-25,01-190722,Jerico,Dos mil,Tradicional,Lavado,3.00,16.666667,2.50,Espressso,08:42:00,175°,195°
2,2022-07-25,01-190722,Jerico,Dos mil,Tradicional,Lavado,9.00,16.444444,7.52,Filtrado,07:58:00,175°,190°
3,2022-07-28,01-190722,Jerico,Dos mil,Tradicional,Lavado,11.70,16.324786,9.79,Filtrado,08:02:00,175°,191°
4,2022-07-28,09-190722,Ciudad Bolivar,Tabi,Natural,Natural,0.45,24.444444,0.34,Filtrado,08:10:00,150°,186°


**Archivo "CC FT 21   Formato de Control de Despachos (1).xlsx"**

En general, este archivo, como su nombre lo indica, contiene información sobre el despacho del café, por lo que no es relevante para nuestro análisis, ya que no tenemos información sobre el Puntaje de Taza. Por lo que directamente no lo utilizaremos en nuestro análisis.

En este punto, necesitamos consolidar la información de los archivos de Control de Calidad y Tostión, para así tener un DataFrame que contenga toda la información relevante para nuestro análisis. Sin embargo, es importante notar ciertos puntos.

- Inconsistencia de las fechas: Las fechas en ambos archivos difieren para un mismo lote y es que, para un mismo lote, la fecha de tostión puede ser diferente a la fecha de control de calidad (en algunas ocasiones, los controles de calidad se realizan después de la tostión, pero sobre un pequeño fragmento ya tostado del lote). Por lo tanto, debemos tener en cuenta que las fechas pueden no coincidir y que debemos relacionar los datos por el lote y no por la fecha. Es por esto que las fechas las vamos a ignorar de momento.

- Agrupación por lotes y suposición de resultados uniformes: Al consolidar los datos, debemos tener en cuenta que los lotes tienen múltiples registros de tostión, a pesar de que solo tienen un Puntaje de Taza. En un análisis inicial, se trató de hacer un mapeo buscando cual de los procesos de tostión era el que se había utilizado para el Puntaje de Taza, sin embargo, no fue posible pues incluso hay algunas catas que tienen tosteos posteriores. Por lo tanto, vamos a asumir que el Puntaje de Taza es el mismo para todos los registros de tostión del mismo lote, sin importar el perfil de tostión utilizado. Esto implica que vamos a consolidar los datos por lote, y vamos a tomar el Puntaje de Taza como el mismo para todos los registros de tostión del mismo lote.

Siendo así, solo resta consolidar los datos de los archivos de Control de Calidad y Tostión, para así tener un DataFrame que contenga toda la información relevante para nuestro análisis. Para esto, vamos a hacer un merge entre los dos DataFrames, utilizando la columna "Lote" como clave de unión. Esto nos permitirá tener un DataFrame que contenga toda la información relevante para nuestro análisis, y que podamos utilizar para entrenar nuestros modelos de regresión.

In [384]:
# Consolidación de los datos de Control de Calidad y Tostión
# Realizamos un merge entre los dos DataFrames, utilizando la columna "Lote" como clave de unión
data_consolidado = pd.merge(data_control_calidad, data_tostion, on='Lote', how='inner')

# Revisamos las primeras filas del DataFrame consolidado
data_consolidado.head()

,Fecha_x,Lote,Denominación,Cantidad,% Humedad,C/NC Humedad,Mallas,C/NC Mallas,Verificación Física Café,Puntaje de Taza,Fecha_y,Origen,Variedad,Proceso,Beneficio,Peso en Verde,Merma,Peso en Tostado,Perfil,Tiempo de tueste,Temp. De inicio,Temp. Final
0,2022-07-19,01-190722,Madre Laura,765.0,10.9,C,14.0,C,C,84,2022-07-22,Jerico,Dos mil,Tradicional,Lavado,9.0,15.000000,7.65,Filtrado,08:01:00,175°,191°
1,2022-07-19,01-190722,Madre Laura,765.0,10.9,C,14.0,C,C,84,2022-07-25,Jerico,Dos mil,Tradicional,Lavado,3.0,16.666667,2.50,Espressso,08:42:00,175°,195°
2,2022-07-19,01-190722,Madre Laura,765.0,10.9,C,14.0,C,C,84,2022-07-25,Jerico,Dos mil,Tradicional,Lavado,9.0,16.444444,7.52,Filtrado,07:58:00,175°,190°
3,2022-07-19,01-190722,Madre Laura,765.0,10.9,C,14.0,C,C,84,2022-07-28,Jerico,Dos mil,Tradicional,Lavado,11.7,16.324786,9.79,Filtrado,08:02:00,175°,191°
4,2022-07-19,01-190722,Madre Laura,765.0,10.9,C,14.0,C,C,84,2022-07-31,Jerico,Dos mil,Tradicional,Lavado,29.0,15.344828,24.55,Filtrado,08:02:00,175°,192°


In [385]:
# Eliminamos las columnas de fechas que no aportan información relevante
data_consolidado = data_consolidado.drop(columns=['Fecha_x', 'Fecha_y'])

# Revisamos las primeras filas del DataFrame consolidado después de eliminar las columnas de fechas
data_consolidado.head()

,Lote,Denominación,Cantidad,% Humedad,C/NC Humedad,Mallas,C/NC Mallas,Verificación Física Café,Puntaje de Taza,Origen,Variedad,Proceso,Beneficio,Peso en Verde,Merma,Peso en Tostado,Perfil,Tiempo de tueste,Temp. De inicio,Temp. Final
0,01-190722,Madre Laura,765.0,10.9,C,14.0,C,C,84,Jerico,Dos mil,Tradicional,Lavado,9.0,15.000000,7.65,Filtrado,08:01:00,175°,191°
1,01-190722,Madre Laura,765.0,10.9,C,14.0,C,C,84,Jerico,Dos mil,Tradicional,Lavado,3.0,16.666667,2.50,Espressso,08:42:00,175°,195°
2,01-190722,Madre Laura,765.0,10.9,C,14.0,C,C,84,Jerico,Dos mil,Tradicional,Lavado,9.0,16.444444,7.52,Filtrado,07:58:00,175°,190°
3,01-190722,Madre Laura,765.0,10.9,C,14.0,C,C,84,Jerico,Dos mil,Tradicional,Lavado,11.7,16.324786,9.79,Filtrado,08:02:00,175°,191°
4,01-190722,Madre Laura,765.0,10.9,C,14.0,C,C,84,Jerico,Dos mil,Tradicional,Lavado,29.0,15.344828,24.55,Filtrado,08:02:00,175°,192°


## Preprocesamiento de los datos

El proceso inicial de preprocesamiento de los datos va a consistir en modificar las columnas categoricas para que sean numéricas. Para esto, recordemos que debemos tener en cuenta el problema de los espacios en los nombres de las filas.

In [386]:
# Revisemos los tipos de datos de las columnas del DataFrame consolidado
data_consolidado.dtypes

Lote                         object
Denominación                 object
Cantidad                    float64
% Humedad                   float64
C/NC Humedad                 object
Mallas                      float64
C/NC Mallas                  object
Verificación Física Café     object
Puntaje de Taza              object
Origen                       object
Variedad                     object
Proceso                      object
Beneficio                    object
Peso en Verde               float64
Merma                       float64
Peso en Tostado             float64
Perfil                       object
Tiempo de tueste             object
Temp. De inicio              object
Temp. Final                  object
dtype: object

In [387]:
# Aplicar strip() solo a columnas tipo objeto (texto)
for col in data_consolidado.select_dtypes(include=['object']).columns:
    data_consolidado[col] = data_consolidado[col].astype(str).str.strip()


In [388]:
data_consolidado.head()

,Lote,Denominación,Cantidad,% Humedad,C/NC Humedad,Mallas,C/NC Mallas,Verificación Física Café,Puntaje de Taza,Origen,Variedad,Proceso,Beneficio,Peso en Verde,Merma,Peso en Tostado,Perfil,Tiempo de tueste,Temp. De inicio,Temp. Final
0,01-190722,Madre Laura,765.0,10.9,C,14.0,C,C,84,Jerico,Dos mil,Tradicional,Lavado,9.0,15.000000,7.65,Filtrado,08:01:00,175°,191°
1,01-190722,Madre Laura,765.0,10.9,C,14.0,C,C,84,Jerico,Dos mil,Tradicional,Lavado,3.0,16.666667,2.50,Espressso,08:42:00,175°,195°
2,01-190722,Madre Laura,765.0,10.9,C,14.0,C,C,84,Jerico,Dos mil,Tradicional,Lavado,9.0,16.444444,7.52,Filtrado,07:58:00,175°,190°
3,01-190722,Madre Laura,765.0,10.9,C,14.0,C,C,84,Jerico,Dos mil,Tradicional,Lavado,11.7,16.324786,9.79,Filtrado,08:02:00,175°,191°
4,01-190722,Madre Laura,765.0,10.9,C,14.0,C,C,84,Jerico,Dos mil,Tradicional,Lavado,29.0,15.344828,24.55,Filtrado,08:02:00,175°,192°


En primer lugar, pasemos la columna de tiempo de tueste a una cantidad de tiempo, en minutos

In [389]:
data_consolidado['Tiempo de tueste'] = pd.to_timedelta(data_consolidado['Tiempo de tueste'])
data_consolidado['Tiempo de tueste (min)'] = data_consolidado['Tiempo de tueste'].dt.total_seconds() / 60
data_consolidado = data_consolidado.drop(columns='Tiempo de tueste')

data_consolidado.head()

,Lote,Denominación,Cantidad,% Humedad,C/NC Humedad,Mallas,C/NC Mallas,Verificación Física Café,Puntaje de Taza,Origen,Variedad,Proceso,Beneficio,Peso en Verde,Merma,Peso en Tostado,Perfil,Temp. De inicio,Temp. Final,Tiempo de tueste (min)
0,01-190722,Madre Laura,765.0,10.9,C,14.0,C,C,84,Jerico,Dos mil,Tradicional,Lavado,9.0,15.000000,7.65,Filtrado,175°,191°,481.0
1,01-190722,Madre Laura,765.0,10.9,C,14.0,C,C,84,Jerico,Dos mil,Tradicional,Lavado,3.0,16.666667,2.50,Espressso,175°,195°,522.0
2,01-190722,Madre Laura,765.0,10.9,C,14.0,C,C,84,Jerico,Dos mil,Tradicional,Lavado,9.0,16.444444,7.52,Filtrado,175°,190°,478.0
3,01-190722,Madre Laura,765.0,10.9,C,14.0,C,C,84,Jerico,Dos mil,Tradicional,Lavado,11.7,16.324786,9.79,Filtrado,175°,191°,482.0
4,01-190722,Madre Laura,765.0,10.9,C,14.0,C,C,84,Jerico,Dos mil,Tradicional,Lavado,29.0,15.344828,24.55,Filtrado,175°,192°,482.0


Además, debemos convertir los datos de las temperaturas de inicio y final a valores numéricos, ya que actualmente están en formato de texto. Esto nos permitirá trabajar con ellos de manera más efectiva en nuestro modelo de regresión.

In [390]:
# Impresión de distintas temperaturas de inicio y final
temperaturas_inicio = data_consolidado['Temp. De inicio'].unique()
temperaturas_final = data_consolidado['Temp. Final'].unique()
print("Temperaturas de inicio únicas:")
print(temperaturas_inicio)
print("Temperaturas finales únicas:")
print(temperaturas_final)

Temperaturas de inicio únicas:
['175°' '175°192°' '150°' '170°' '180°']
Temperaturas finales únicas:
['191°' '195°' '190°' '192°' '196°' '193°' '192' '195' '196' '194°' 'None'
 '186°' '189°']


Aquí encontramos un problema en un dato específico que no tenía la división /, por lo que debemos corregirlo manualmente. Esto es importante para evitar errores en el análisis posterior.

In [391]:
# Obtenemos el dato específico que no tenía la división /
data_consolidado['Temp. De inicio'] = data_consolidado['Temp. De inicio'].replace('175°192°', '175°')
data_consolidado['Temp. Final'] = data_consolidado['Temp. Final'].replace('None', '192°')

In [392]:
# Eliminar el símbolo "°C" y convertir a float
data_consolidado['Temp. De inicio'] = data_consolidado['Temp. De inicio'].str.replace('°', '').astype(float)
data_consolidado['Temp. Final'] = data_consolidado['Temp. Final'].str.replace('°', '').astype(float)

# Convertir las columnas de temperatura a tipo float
data_consolidado.head()

,Lote,Denominación,Cantidad,% Humedad,C/NC Humedad,Mallas,C/NC Mallas,Verificación Física Café,Puntaje de Taza,Origen,Variedad,Proceso,Beneficio,Peso en Verde,Merma,Peso en Tostado,Perfil,Temp. De inicio,Temp. Final,Tiempo de tueste (min)
0,01-190722,Madre Laura,765.0,10.9,C,14.0,C,C,84,Jerico,Dos mil,Tradicional,Lavado,9.0,15.000000,7.65,Filtrado,175.0,191.0,481.0
1,01-190722,Madre Laura,765.0,10.9,C,14.0,C,C,84,Jerico,Dos mil,Tradicional,Lavado,3.0,16.666667,2.50,Espressso,175.0,195.0,522.0
2,01-190722,Madre Laura,765.0,10.9,C,14.0,C,C,84,Jerico,Dos mil,Tradicional,Lavado,9.0,16.444444,7.52,Filtrado,175.0,190.0,478.0
3,01-190722,Madre Laura,765.0,10.9,C,14.0,C,C,84,Jerico,Dos mil,Tradicional,Lavado,11.7,16.324786,9.79,Filtrado,175.0,191.0,482.0
4,01-190722,Madre Laura,765.0,10.9,C,14.0,C,C,84,Jerico,Dos mil,Tradicional,Lavado,29.0,15.344828,24.55,Filtrado,175.0,192.0,482.0


Ahora si, codifiquemos las columnas categóricas para que sean numéricas, y así poder utilizarlas en nuestro modelo de regresión. Para esto, vamos a utilizar la función `labelencoder()` de sklearn, que nos permite convertir las columnas categóricas en variables numéricas que podemos utilizar en nuestro modelo de regresión.

In [393]:
# Tipos de las columnas
print(data_consolidado.dtypes)

Lote                         object
Denominación                 object
Cantidad                    float64
% Humedad                   float64
C/NC Humedad                 object
Mallas                      float64
C/NC Mallas                  object
Verificación Física Café     object
Puntaje de Taza              object
Origen                       object
Variedad                     object
Proceso                      object
Beneficio                    object
Peso en Verde               float64
Merma                       float64
Peso en Tostado             float64
Perfil                       object
Temp. De inicio             float64
Temp. Final                 float64
Tiempo de tueste (min)      float64
dtype: object


Como vemos, debemos convertir la columna de "Puntaje de Taza" a una variable numérica, ya que actualmente está en formato de texto.

In [394]:
# Convertir la columna "Puntaje de Taza" a numérico
data_consolidado['Puntaje de Taza'] = pd.to_numeric(data_consolidado['Puntaje de Taza'], errors='coerce')

data_consolidado.head()

,Lote,Denominación,Cantidad,% Humedad,C/NC Humedad,Mallas,C/NC Mallas,Verificación Física Café,Puntaje de Taza,Origen,Variedad,Proceso,Beneficio,Peso en Verde,Merma,Peso en Tostado,Perfil,Temp. De inicio,Temp. Final,Tiempo de tueste (min)
0,01-190722,Madre Laura,765.0,10.9,C,14.0,C,C,84.0,Jerico,Dos mil,Tradicional,Lavado,9.0,15.000000,7.65,Filtrado,175.0,191.0,481.0
1,01-190722,Madre Laura,765.0,10.9,C,14.0,C,C,84.0,Jerico,Dos mil,Tradicional,Lavado,3.0,16.666667,2.50,Espressso,175.0,195.0,522.0
2,01-190722,Madre Laura,765.0,10.9,C,14.0,C,C,84.0,Jerico,Dos mil,Tradicional,Lavado,9.0,16.444444,7.52,Filtrado,175.0,190.0,478.0
3,01-190722,Madre Laura,765.0,10.9,C,14.0,C,C,84.0,Jerico,Dos mil,Tradicional,Lavado,11.7,16.324786,9.79,Filtrado,175.0,191.0,482.0
4,01-190722,Madre Laura,765.0,10.9,C,14.0,C,C,84.0,Jerico,Dos mil,Tradicional,Lavado,29.0,15.344828,24.55,Filtrado,175.0,192.0,482.0


In [395]:
# Crear un LabelEncoder
le = LabelEncoder()

# Aplicar a todas las columnas categóricas
for col in data_consolidado.select_dtypes(include=['object']).columns:
    data_consolidado[col] = le.fit_transform(data_consolidado[col])

In [396]:
data_consolidado.head()

,Lote,Denominación,Cantidad,% Humedad,C/NC Humedad,Mallas,C/NC Mallas,Verificación Física Café,Puntaje de Taza,Origen,Variedad,Proceso,Beneficio,Peso en Verde,Merma,Peso en Tostado,Perfil,Temp. De inicio,Temp. Final,Tiempo de tueste (min)
0,4,12,765.0,10.9,0,14.0,0,0,84.0,4,3,3,2,9.0,15.000000,7.65,2,175.0,191.0,481.0
1,4,12,765.0,10.9,0,14.0,0,0,84.0,4,3,3,2,3.0,16.666667,2.50,1,175.0,195.0,522.0
2,4,12,765.0,10.9,0,14.0,0,0,84.0,4,3,3,2,9.0,16.444444,7.52,2,175.0,190.0,478.0
3,4,12,765.0,10.9,0,14.0,0,0,84.0,4,3,3,2,11.7,16.324786,9.79,2,175.0,191.0,482.0
4,4,12,765.0,10.9,0,14.0,0,0,84.0,4,3,3,2,29.0,15.344828,24.55,2,175.0,192.0,482.0


Como podemos ver, ya tenemos variables numéricas que podemos utilizar en nuestros modelos de regresión. Sin embargo, convendrá normalizar algunas de estas variables, ya que algunas tienen un rango de valores mucho mayor que otras, lo que puede afectar el rendimiento del modelo. Por ejemplo, las temperaturas de inicio y final tienen un rango de valores mucho mayor que las demás variables, por lo que convendrá normalizarlas para evitar problemas de escala.

In [399]:
# Lista de columnas numéricas para normalizar
columnas_a_normalizar = ['Cantidad', 'Peso en Verde', 'Merma', 'Peso en Tostado', 'Temp. De inicio', 'Temp. Final', 'Tiempo de tueste (min)', '% Humedad', 'Mallas']

# Generar un objeto MinMaxScaler
scaler = MinMaxScaler()
# Normalizar las columnas seleccionadas
data_consolidado[columnas_a_normalizar] = scaler.fit_transform(data_consolidado[columnas_a_normalizar])

# Mostramos las primeras filas del DataFrame consolidado después de la normalización
data_consolidado.head()

,Lote,Denominación,Cantidad,% Humedad,C/NC Humedad,Mallas,C/NC Mallas,Verificación Física Café,Puntaje de Taza,Origen,Variedad,Proceso,Beneficio,Peso en Verde,Merma,Peso en Tostado,Perfil,Temp. De inicio,Temp. Final,Tiempo de tueste (min)
0,4,12,1.0,0.7,0,0.0,0,0,84.0,4,3,3,2,0.042846,0.210165,0.043731,2,0.833333,0.5,0.702586
1,4,12,1.0,0.7,0,0.0,0,0,84.0,4,3,3,2,0.012779,0.313187,0.012922,1,0.833333,0.9,0.879310
2,4,12,1.0,0.7,0,0.0,0,0,84.0,4,3,3,2,0.042846,0.299451,0.042953,2,0.833333,0.4,0.689655
3,4,12,1.0,0.7,0,0.0,0,0,84.0,4,3,3,2,0.056377,0.292054,0.056533,2,0.833333,0.5,0.706897
4,4,12,1.0,0.7,0,0.0,0,0,84.0,4,3,3,2,0.143072,0.231480,0.144831,2,0.833333,0.6,0.706897


Este DataFrame consolidado contiene toda la información relevante para nuestro análisis, y ya hemos realizado el preprocesamiento inicial de los datos. Ahora podemos proceder a entrenar nuestros modelos de regresión y evaluar su rendimiento.